In [1]:
import time
import numpy as np
import pandas as pd
import gc
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
act_1 = pd.read_hdf('../input/act_1_features.h5')
act_1.head()

,item_id,course_id,watching_count_sum,watching_count_count,watching_count_mean,watching_count_max,watching_count_min,watching_count_std,watching_count_skew,video_duration_sum,video_duration_count,video_duration_mean,video_duration_max,video_duration_min,video_duration_std,video_duration_skew,local_watching_time_sum,local_watching_time_count,local_watching_time_mean,local_watching_time_max,local_watching_time_min,local_watching_time_std,local_watching_time_skew,video_progress_time_sum,video_progress_time_count,video_progress_time_mean,video_progress_time_max,video_progress_time_min,video_progress_time_std,video_progress_time_skew,video_watching_duration_sum,video_watching_duration_count,video_watching_duration_mean,video_watching_duration_max,video_watching_duration_min,video_watching_duration_std,video_watching_duration_skew,local_watching_duration_sum,local_watching_duration_count,local_watching_duration_mean,local_watching_duration_max,local_watching_duration_min,local_watching_duration_std,local_watching_duration_skew,video_is_end_sum,video_is_end_count,video_is_end_mean,video_is_end_max,video_is_end_min,video_is_end_std,video_is_end_skew,video_is_start_sum,video_is_start_count,video_is_start_mean,video_is_start_max,video_is_start_min,video_is_start_std,video_is_start_skew,video_is_start_end_sum,video_is_start_end_count,video_is_start_end_mean,video_is_start_end_max,video_is_start_end_min,video_is_start_end_std,video_is_start_end_skew,forward_seconds_sum,forward_seconds_count,forward_seconds_mean,forward_seconds_max,forward_seconds_min,forward_seconds_std,forward_seconds_skew,local_watching_diff_sum,local_watching_diff_count,local_watching_diff_mean,local_watching_diff_max,local_watching_diff_min,local_watching_diff_std,local_watching_diff_skew,speed_sum,speed_count,speed_mean,speed_max,speed_min,speed_std,speed_skew,course_id_count,local_watching_duration,local_watching_diff,local_watching_video_duration,label
0,V_0,C_course-v1:GXUST+2017041901X+2017_T1,2,2,1.000000,1,1,0.000000,NaN,1069.0,2,534.500000,601.0,468.0,94.045202,NaN,1079,2,539.5,600,479,85.559921,NaN,1069.000000,2,534.500000,601.000000,468.000000,94.045202,NaN,1069.000000,2,534.500000,601.000000,468.000000,94.045202,NaN,1079,2,539.500000,600,479,85.559921,NaN,0.000000,2,0.000000,0.000000,0.000000,0.000000,NaN,0,2,0.000000,0,0,0.00000,NaN,0.000000,2,0.000000,0.000000,0.000000,0.000000,NaN,0.000000e+00,2,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,0,2,0.000000,0,0,0.000000,NaN,2.021837,2,1.010918,1.023502,0.998334,0.017796,NaN,8,1196,117,127.0,0
1,V_0,C_course-v1:TsinghuaX+00310222X+sp,3,2,1.500000,2,1,0.707107,NaN,1899.0,2,949.500000,1132.0,767.0,258.093975,NaN,879,2,439.5,774,105,473.054437,NaN,871.230011,2,435.615005,767.000000,104.230011,468.649154,NaN,998.550003,2,499.275002,767.000000,231.550003,378.620324,NaN,879,2,439.500000,774,105,473.054437,NaN,-900.449997,2,-450.224998,0.000000,-900.449997,636.714299,NaN,0,2,0.000000,0,0,0.00000,NaN,-900.449997,2,-450.224998,0.000000,-900.449997,636.714299,NaN,1.273200e+02,2,6.366000e+01,1.273200e+02,0.000000,9.002883e+01,NaN,0,2,0.000000,0,0,0.000000,NaN,2.016503,2,1.008251,1.009125,1.007378,0.001236,NaN,8,53088,52209,51189.0,1
2,V_0,C_course-v1:TsinghuaX+00740043X_2015_T2+sp,1,1,1.000000,1,1,NaN,NaN,937.0,1,937.000000,937.0,937.0,NaN,NaN,223,1,223.0,223,223,NaN,NaN,222.869995,1,222.869995,222.869995,222.869995,NaN,NaN,222.869995,1,222.869995,222.869995,222.869995,NaN,NaN,223,1,223.000000,223,223,NaN,NaN,-714.130005,1,-714.130005,-714.130005,-714.130005,NaN,NaN,0,1,0.000000,0,0,NaN,NaN,-714.130005,1,-714.130005,-714.130005,-714.130005,NaN,NaN,0.000000e+00,1,0.000000e+00,0.000000e+00,0.000000,NaN,NaN,0,1,0.000000,0,0,NaN,NaN,1.000579,1,1.000579,1.000579,1.000579,NaN,NaN,8,223,0,-714.0,0
3,V_0,C_course-v1:TsinghuaX+30240184+sp,5,3,1.666667,3,1,1.154701,1.732051,730.0,3,243.333333,288.0,209.0,40.501029,1.073382,1035,3,345.0,613,186,233.437358,1.643084,729.280000,3,243.093333,287.280000,209.000000,40.104191,1.060

In [3]:
act_1.shape

(187207, 91)

In [4]:
act_2 = pd.read_hdf('../input/act_2_features.h5')
act_2.head()

,item_id,course_id,watching_count_sum,watching_count_count,watching_count_mean,watching_count_max,watching_count_min,watching_count_std,watching_count_skew,video_duration_sum,video_duration_count,video_duration_mean,video_duration_max,video_duration_min,video_duration_std,video_duration_skew,local_watching_time_sum,local_watching_time_count,local_watching_time_mean,local_watching_time_max,local_watching_time_min,local_watching_time_std,local_watching_time_skew,video_progress_time_sum,video_progress_time_count,video_progress_time_mean,video_progress_time_max,video_progress_time_min,video_progress_time_std,video_progress_time_skew,video_watching_duration_sum,video_watching_duration_count,video_watching_duration_mean,video_watching_duration_max,video_watching_duration_min,video_watching_duration_std,video_watching_duration_skew,local_watching_duration_sum,local_watching_duration_count,local_watching_duration_mean,local_watching_duration_max,local_watching_duration_min,local_watching_duration_std,local_watching_duration_skew,video_is_end_sum,video_is_end_count,video_is_end_mean,video_is_end_max,video_is_end_min,video_is_end_std,video_is_end_skew,video_is_start_sum,video_is_start_count,video_is_start_mean,video_is_start_max,video_is_start_min,video_is_start_std,video_is_start_skew,video_is_start_end_sum,video_is_start_end_count,video_is_start_end_mean,video_is_start_end_max,video_is_start_end_min,video_is_start_end_std,video_is_start_end_skew,forward_seconds_sum,forward_seconds_count,forward_seconds_mean,forward_seconds_max,forward_seconds_min,forward_seconds_std,forward_seconds_skew,local_watching_diff_sum,local_watching_diff_count,local_watching_diff_mean,local_watching_diff_max,local_watching_diff_min,local_watching_diff_std,local_watching_diff_skew,speed_sum,speed_count,speed_mean,speed_max,speed_min,speed_std,speed_skew,course_id_count,local_watching_duration,local_watching_diff,local_watching_video_duration,label
0,U_0,C_course-v1:TsinghuaX+00691153X+sp,14,8,1.750000,4,1,1.164965,1.355361,4867.0,8,608.375000,929.0,282.0,264.815266,0.014851,3982,8,497.750000,930,202,269.890851,0.720634,3976.209992,8,497.026249,929.000000,201.529999,269.829949,0.723786,3944.460001,8,493.057500,929.000000,201.529999,273.672031,0.694879,19541,8,2442.625000,15377,202,5232.548490,2.814853,-922.529999,8,-115.316250,0.000000,-696.470001,241.414980,-2.553037,1,8,0.125000,1,0,0.353553,2.828427,-922.539999,8,-115.317500,0.000000,-696.470001,241.414711,-2.553025,-3.174999e+01,8,-3.968749e+00,1.900024e-01,-2.771001e+01,9.708359e+00,-2.705540,15559,8,1944.875000,14962,0,5263.090657,2.821294,8.013482,8,1.001685,1.003349,0.999999,0.001311,0.090651,36,15882,11900,11015.0,0
1,U_0,C_course-v1:TsinghuaX+10421075X_2015_2+sp,2,1,2.000000,2,2,NaN,NaN,748.0,1,748.000000,748.0,748.0,NaN,NaN,192,1,192.000000,192,192,NaN,NaN,191.349991,1,191.349991,191.349991,191.349991,NaN,NaN,191.539993,1,191.539993,191.539993,191.539993,NaN,NaN,256,1,256.000000,256,256,NaN,NaN,-556.460007,1,-556.460007,-556.460007,-556.460007,NaN,NaN,0,1,0.000000,0,0,NaN,NaN,-556.460007,1,-556.460007,-556.460007,-556.460007,NaN,NaN,1.900024e-01,1,1.900024e-01,1.900024e-01,1.900024e-01,NaN,NaN,64,1,64.000000,64,64,NaN,NaN,1.003392,1,1.003392,1.003392,1.003392,NaN,NaN,36,256,64,-492.0,0
2,U_0,C_course-v1:TsinghuaX+20430064X+sp,14,14,1.000000,1,1,0.000000,0.000000,5381.0,14,384.357143,896.0,173.0,191.134913,1.525479,4584,14,327.428571,563,93,139.177095,0.194791,4577.169999,14,326.940714,563.000000,92.209999,139.322856,0.196968,4577.169999,14,326.940714,563.000000,92.209999,139.322856,0.196968,4584,14,327.428571,563,93,139.177095,0.194791,-803.790001,14,-57.413571,0.000000,-803.790001,214.821914,-3.741657,4,14,0.285714,1,0,0.468807,1.066536,-803.830001,14,-57.416429,0.000000,-803.790001,214.821092,-3.741657,1.705303e-13,14,1.218073e-14,5.684342e-14,0.000000e+00,2.420480e-14,0.000000,0,14,0.000000,0,0,0.000000,0.000000,14.029990,14,1.002142,1.008556,0.999995,0.002872,1.048917,36,5853,1269,472.0,1
3,U_0,C

In [5]:
act_2.shape

(40727, 91)

In [6]:
train = pd.concat([act_1, act_2], ignore_index=False, axis=0)
train.drop(['item_id', 'course_id'], axis=1, inplace=True)
train.shape

(227934, 89)

In [7]:
test = pd.read_hdf('../input/test_features.h5')
submit = test[['item_id', 'course_id']]
test.drop(['item_id', 'course_id'], axis=1, inplace=True)
test.shape

(20023, 88)

In [8]:
y = train['label']
train.drop('label', axis=1, inplace=True)
train.shape

(227934, 88)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, stratify=y, random_state=2020, shuffle=True)

In [10]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)
lgb_all = lgb.Dataset(train, y, reference=lgb_train)

In [11]:
params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'binary_error',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'min_data_in_leaf': 20,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}

In [12]:
lgb_model_valid = lgb.train(params,
                            lgb_train,
                            valid_sets=[lgb_train, lgb_valid],
                            early_stopping_rounds=500,
                            num_boost_round=100000,
                            verbose_eval=500)

Training until validation scores don't improve for 500 rounds
[500]	training's binary_error: 0.171793	valid_1's binary_error: 0.209129
[1000]	training's binary_error: 0.131846	valid_1's binary_error: 0.200284
[1500]	training's binary_error: 0.103463	valid_1's binary_error: 0.195599
[2000]	training's binary_error: 0.0816321	valid_1's binary_error: 0.190878
[2500]	training's binary_error: 0.066148	valid_1's binary_error: 0.188807
[3000]	training's binary_error: 0.0544955	valid_1's binary_error: 0.186789
[3500]	training's binary_error: 0.0445393	valid_1's binary_error: 0.18528
[4000]	training's binary_error: 0.03673	valid_1's binary_error: 0.18421
[4500]	training's binary_error: 0.0302954	valid_1's binary_error: 0.182823
[5000]	training's binary_error: 0.024165	valid_1's binary_error: 0.182367
[5500]	training's binary_error: 0.0196373	valid_1's binary_error: 0.181384
[6000]	training's binary_error: 0.0160105	valid_1's binary_error: 0.180665
[6500]	training's binary_error: 0.0132202	valid_

In [13]:
y_prob = lgb_model_valid.predict(X_valid)
y_pred = np.where(y_prob > 0.5, 1, 0)

acc = np.round(accuracy_score(y_valid, y_pred), 4)
auc = np.round(roc_auc_score(y_valid, y_prob), 4)

print('acc: ', acc)
print('auc: ', auc)

acc:  0.8222
auc:  0.8893


In [14]:
importance = lgb_model_valid.feature_importance(importance_type='gain')
feature_name = lgb_model_valid.feature_name()
feature_importance = pd.DataFrame({'feature_name': feature_name, 'importance': importance}).sort_values(by='importance', ascending=False)
feature_importance.to_csv('../input/feature_importance.csv', index=False)

In [15]:
lgb_model = lgb.train(params,
                      lgb_all,
                      num_boost_round=lgb_model_valid.best_iteration + 100)

In [16]:
submit['label'] = np.where(lgb_model.predict(test) > 0.5, 1, 0).astype(int)

In [17]:
test_label = pd.read_hdf('../input/test_label.h5')
test_label = test_label.merge(submit, how='left', on=['item_id', 'course_id'])
test_label.head()

,item_id,course_id,label
0,T_1,C_course-v1:TsinghuaX+00612642X+sp,0
1,T_1,C_course-v1:TsinghuaX+80000271X+sp,0
2,T_1,C_course-v1:TsinghuaX+00701032X+sp,0
3,T_1,C_course-v1:TsinghuaX+80000901X_1+sp,0
4,T_2,C_course-v1:GZHU+136300904+sp,0


In [18]:
def fun1(dx):
    return list(dx.values)

sub = test_label.groupby('item_id')['label'].apply(fun1).reset_index()
sub.columns = ['item_id', 'label_list']
sub = sub[['label_list', 'item_id']]

sub.to_json('../sub/sub_{}_{}.json'.format(time.strftime('%Y%m%d'), str(acc)), orient='records', lines=True)